In [1]:
import geemap
import os
import ee
geemap.set_proxy(port=7890)

In [2]:
Map=geemap.Map(center=(40,100),zoom=4)
Map

Map(center=[40, 100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=F…

In [ ]:
geemap.ee_search()

In [3]:
roi = ee.FeatureCollection('users/cugbxwd/resize')
Map.addLayer(roi, {}, "resize")
Map.centerObject(roi, 7)

In [4]:
dataset = ee.ImageCollection('LANDSAT/LC08/C01/T1_RT')\
                  .filterDate('2017-01-01', '2017-12-31')\
                .filterBounds(roi)
dataset_vis={
    'min': 0.0,
    'max': 30000.0,
    'bands':['B4', 'B3', 'B5']
}
Map.addLayer(dataset,dataset_vis,"landsat8")

In [5]:
images=dataset.median()
collections=images.clipToCollection(roi)
dataset_vis={
    'min': 0.0,
    'max': 30000.0,
    'bands':['B4', 'B3', 'B5']
}
Map.addLayer(collections,dataset_vis,"collections")

In [ ]:
images = geemap.landsat_timeseries(roi=roi, start_year=1991, end_year=2020, start_date='06-01', end_date='09-30')

In [9]:
first_image = ee.Image(dataset.toList(dataset.size()).get(0))

In [10]:
dataset_vis={
    'min': 0.0,
    'max': 30000.0,
    'bands':['B4', 'B3', 'B5']
}
Map.addLayer(first_image, dataset_vis, 'First image')


In [6]:
ndwi_threshold = 0
def extract_water(img):
    
    ndwi_image = img.normalizedDifference(['B3', 'B5'])
    water_image = ndwi_image.gt(ndwi_threshold)
    return water_image

In [7]:
water_images = dataset.map(extract_water)

In [11]:
first_water_image = ee.Image(water_images.toList(water_images.size()).get(0)).selfMask()
Map.addLayer(first_water_image, {'palette': 'B2'}, 'First NDWI')

EEException: Image.visualize: Color is not a valid CSS 3.0 color ('FF0000' or 'red' for red). Found: '#B2'.

In [ ]:
first_image = ee.Image(images.toList(images.size()).get(0))
landsat_vis = {
    'bands': ['NIR', 'Red', 'Green'],
    'min': 0, 
    'max': 3500
}
Map.addLayer(first_image, landsat_vis, 'First image')

first_water_image = ee.Image(water_images.toList(water_images.size()).get(0)).selfMask()
Map.addLayer(first_water_image, {'palette': 'blue'}, 'First NDWI')



In [ ]:
video_args = {
  'dimensions': 768,
  'region': roi,
  'framesPerSecond': 100,
  
  'min': 0,
  'max': 4000,

}

In [ ]:
work_dir = os.path.join('E:\\geemap\\downloads\\', 'Downloads')
if not os.path.exists(work_dir):
    os.makedirs(work_dir)
out_gif = os.path.join(work_dir, "surfacewater.gif")

In [ ]:
geemap.download_ee_video( water_images, video_args, out_gif)

In [ ]:
layer_index = 0  # Change this index to your desired date
print(roi[layer_index])

water_image = ee.Image(water_images.toList(water_images.size()).get(layer_index)).selfMask()
landsat_image = ee.Image(images.toList(images.size()).get(layer_index))

water_vis = {
    'palette': 'blue'
}

landsat_vis = {
  'bands': ['NIR', 'Red', 'Green'],
  'min': 0,
  'max': 4000,
  'gamma': 1.4,
}

left_layer = geemap.ee_tile_layer(water_image, water_vis, 'Water')
right_layer = geemap.ee_tile_layer(landsat_image, landsat_vis, 'Landsat')

Map = geemap.Map()
Map.addLayer(landsat_image, landsat_vis, 'Landsat')
Map.split_map(left_layer, right_layer)
Map.centerObject(roi, zoom=12)
Map